# Radial Basis Functions a Recurrent Neural Network 

Minule jsme si ukazovali a zkoušeli si naprogramovat jednoduché neuronové sítě. Dneska se podíváme na složitější struktury neuronových sítí Radial Basis Functions a rekurentní neuronové sítě.


In [1]:
import random
import sys

import numpy as np

from sklearn import datasets
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split

import tensorflow as tf

## Radial Basis Functions (RBF)

Naimplementujeme si nyní RBF síť. Implementace je jednoduchá - stačí naimplementovat třídu podobnou té níže, kde vlastní výpočet je schován do funkce *predict*, trénování klasicky do funkce *fit* a to nejdůležitější, výpočet RBF aktivace, pak do metody *calculate_activation*.

In [ ]:
class RBFNetwork():
    def __init__(self, input_dim, num_centers, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_centers = num_centers
        self.centers = [np.random.uniform(-1, 1, input_dim) for i in range(num_centers)]
        self.beta = 1
        self.weights = None
        
        
    # Compute values of neurons in the hidden layer
    def calculate_activation(self, data):
        hidden_layer_values = np.zeros((data.shape[0], self.num_centers), float)
        for c_idx, c in enumerate(self.centers):
            hidden_layer_values[:, c_idx] = self.activation_fcn(data, c, self.beta)
        
        return hidden_layer_values
    
    
    # Value of the activation function (for the hidden - RBF - layer)
    def activation_fcn(self, data, center, beta):
        return np.exp(-beta * np.linalg.norm(data - center, axis=-1)**2)
    
    
    def fit(self, data, labels):
        # PART 1 - Set centers for the first layer
        
        # Choose random values (from the dataset) for the initial centers
        random_idx = np.random.permutation(data.shape[0])[:self.num_centers]
        self.centers = [data[i,:] for i in random_idx]
        
        # PART 2 - Set weights for the second layer
         
        # Calculate activation on the hidden layer
        hidden_layer_values = self.calculate_activation(data)
         
        # Compare the real and the predicted outputs and update our weights
        # (Pseudoinverse matrix basically corresponds to the formula for linear
        # regression we would use for training of the weights.)
        self.weights = np.dot(np.linalg.pinv(hidden_layer_values), labels)
          
          
    def predict(self, data):
        hidden_layer_values = self.calculate_activation(data)
        labels = np.dot(hidden_layer_values, self.weights)
        return labels


Zkusíme si naši RBF síť pustit na našem oblíbeném datasetu Iris. Načteme si data a labely, do které třídy data patří. Protože budeme dělat klasifikaci, je vhodné si labely převést na one-hot-encoding. Následně data rozdělíme na trénovací a testovací množinu, abychom mohli zvlášť data trénovat a zvlášť data testovat.

In [3]:
# Get data
iris = datasets.load_iris()
x, y = iris.data, iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# Format the input using one-hot-encoding
# -> Create a ColumnTransformer that will apply OneHotEncoder to the first column
column_transformer = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(), [0])
    ],
    remainder="passthrough" # This will pass the rest of the columns through without any transformation
)

# Apply the ColumnTransformer to y_train and y_test
y_train_onehot = column_transformer.fit_transform(y_train.reshape(-1, 1))
y_test_onehot = column_transformer.transform(y_test.reshape(-1, 1))

# Train and test the network multiple times
for _ in range(10):
    rbf = RBFNetwork(4, 10, 3)
    rbf.fit(x_train, y_train_onehot)
    predicted = rbf.predict(x_test)
    y_pred = np.argmax(predicted, axis=1)
    accuracy = np.mean(y_pred == y_test)
    print("Accuracy: " + str(accuracy))

Accuracy: 0.9473684210526315
Accuracy: 0.9210526315789473
Accuracy: 0.9473684210526315
Accuracy: 0.9473684210526315
Accuracy: 0.9473684210526315
Accuracy: 1.0
Accuracy: 0.9473684210526315
Accuracy: 0.9473684210526315
Accuracy: 0.9210526315789473
Accuracy: 0.9210526315789473


Když si trénování a predikci spustíme několikrát, vidíme, že accuracy je v každém běhu dosti různá, což je způsobeno tím, že centroidy a betu zde nastavujeme náhodně, přestože jsme si říkali, že pozice centroidů se dá trénovat pomocí algoritmu k-means.


### Úkol na cvičení

Zkuste si naimplementovat algoritmus k-means pro inicializaci středů vstupních neuronů a zlepšit tím výstup sítě výše. Stejně tak zkuste si trénovat i bety (viz látka na cvičení). A zkuste si, jak Vám pomůže preprocessing dat (třeba normalizace na střední hodnotu 0 a rozptyl 1, nebo převedení do intervalu [0;1]).

## Rekurentní neuronové sítě (RNN)

Rekurentní neuronová síť je síť, která navíc ke svému vstupu ještě bere jako další vstup svůj výstup z předchozího kroku. Proto výstupy z předchozích výpočtů mohou ovlivňovat výpočty následující, což se může hodit například u generování časových řad nebo textu. Nejprve se podíváme, jak by zhruba mohla vypadat implementace jednoduché RNN, kdybychom si ji psali celou sami. 

Vytvoříme si nejprve jednoduché věty a budeme chtít, aby nám naše sít uměla predikovat, zda je daná věta pozitivní, nebo negativní. 

In [5]:
train_data = {
  "good": True,
  "bad": False,
  "happy": True,
  "sad": False,
  "not good": False,
  "not bad": True,
  "not happy": False,
  "not sad": True,
  "very good": True,
  "very bad": False,
  "very happy": True,
  "very sad": False,
  "i am happy": True,
  "this is good": True,
  "i am bad": False,
  "this is bad": False,
  "i am sad": False,
  "this is sad": False,
  "i am not happy": False,
  "this is not good": False,
  "i am not bad": True,
  "this is not sad": True,
  "i am very happy": True,
  "this is very good": True,
  "i am very bad": False,
  "this is very sad": False,
  "this is very happy": True,
  "i am good not bad": True,
  "this is good not bad": True,
  "i am bad not good": False,
  "i am good and happy": True,
  "this is not good and not happy": False,
  "i am not at all good": False,
  "i am not at all bad": True,
  "i am not at all happy": False,
  "this is not at all sad": True,
  "this is not at all happy": False,
  "i am good right now": True,
  "i am bad right now": False,
  "this is bad right now": False,
  "i am sad right now": False,
  "i was good earlier": True,
  "i was happy earlier": True,
  "i was bad earlier": False,
  "i was sad earlier": False,
  "i am very bad right now": False,
  "this is very good right now": True,
  "this is very sad right now": False,
  "this was bad earlier": False,
  "this was very good earlier": True,
  "this was very bad earlier": False,
  "this was very happy earlier": True,
  "this was very sad earlier": False,
  "i was good and not bad earlier": True,
  "i was not good and not happy earlier": False,
  "i am not at all bad or sad right now": True,
  "i am not at all good or happy right now": False,
  "this was not happy and not good earlier": False,
}

test_data = {
  "this is happy": True,
  "i am good": True,
  "this is not happy": False,
  "i am not good": False,
  "this is not bad": True,
  "i am not sad": True,
  "i am very good": True,
  "this is very bad": False,
  "i am very sad": False,
  "this is bad not good": False,
  "this is good and happy": True,
  "i am not good and not happy": False,
  "i am not at all sad": True,
  "this is not at all good": False,
  "this is not at all bad": True,
  "this is good right now": True,
  "this is sad right now": False,
  "this is very bad right now": False,
  "this was good earlier": True,
  "i was not happy and not good earlier": False,
}

Nejprve je potřeba udělat nějaký preprocessing vět. Tím se myslí převést text do číselné reprezentace. To uděláme třeba tak, že najdeme všechna unikátní slova, očíslujeme je, a každé slovo pak nahradíme jeho číslem. Následně uděláme one-hot-encoding každého slova (a to potom bude mít shape (vocab_size, 1)). 

In [6]:
# Create a vocabulary containing every word in the data
vocab = list(set(w for text in train_data.keys() for w in text.split()))
vocab_size = len(vocab)
print(vocab)
print("Unique words: " + str(vocab_size))

# Replace every word by its index
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }
print(word_to_idx["happy"]) 
print(idx_to_word[0]) 


# Convert to one-hot-encoding
def create_inputs(text):
    inputs = []
    for w in text.split():
        v = np.zeros((vocab_size, 1))
        v[word_to_idx[w]] = 1
        inputs.append(v)
    return inputs

['right', 'is', 'very', 'all', 'good', 'sad', 'this', 'now', 'and', 'at', 'or', 'bad', 'am', 'happy', 'was', 'not', 'earlier', 'i']
Unique words: 18
13
right


Nyní si napíšeme samotnou RNN. Informace z předchozího kroku se ukládá v RNN buňce do speciální proměnné - tzv. skrytého stavu, který nám pak bude ovlivňovat další výstupy a bude se s každým novým vstupem v každém kroku aktualizovat. Aktuální skrytý stav se počítá podle předchozího skrytého stavu a aktuálního vstupu. Výstup je spočítán pomocí aktuálního skrytého stavu. Pro každý krok se používají stejné 3 váhové matice: pro spoje z aktuálních vstupů do aktuálních skrytých, pro spoje z předchozích skrytých do aktuálních skrytých a pro spoje z aktuálních skrytých do výstupů. Zároveň potřebujeme bias pro spočtení skrytého stavu a další pro spočtení výstupu. Pomocí aktivační funkce tanh a dosazení hodnot do rovnice vypočteme výstup a update skrytých stavů.

In [7]:
class RNN:
    def __init__(self, input_size, output_size, hidden_size=64):
        # Weights - the division is used for lowering the variance
        self.Whh = np.random.randn(hidden_size, hidden_size) / 1000
        self.Wxh = np.random.randn(hidden_size, input_size) / 1000
        self.Why = np.random.randn(output_size, hidden_size) / 1000

        # Biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
        
    # Processes whole sequence (not just one token / piece)
    def forward(self, inputs):
        # Variable for remembering the previous hidden state value
        h = np.zeros((self.Whh.shape[0], 1))

        # Gradual update of the hidden state
        for x in inputs:
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)

        # Output vector computation
        y = np.dot(self.Why, h) + self.by
        return y, h

In [8]:
# Let us define function softmax for mapping values to interval [0,1]
def softmax(x):
    return np.exp(x) / sum(np.exp(x))


# RNN initialization
inputs = create_inputs("i am very good")
rnn = RNN(vocab_size, 2)
y, h = rnn.forward(inputs)
probs = softmax(y)
print(probs)

[[0.49999555]
 [0.50000445]]


Vidíme, že síť nám sice nějak funguje, ale není moc užitečná. Problém je, že nijak netrénujeme váhy. K tomu je potřeba si definovat ztrátovou (loss) funkci. Použijeme [cross-entropy loss](https://en.wikipedia.org/wiki/Cross-entropy#Cross-entropy_loss_function_and_logistic_regression), která se v tomto případě spočítá pro každý vstup v podstatě jako minus logaritmus pravděpodobnosti, s jakou predikuje náš model tu opravdovou třídu (label) daného vstupu. Zároveň je potřeba dopsat zpětnou propagaci chyby, aby se síť mohla učit ze svých chyb a updatovat si váhy a skryté stavy. To je v podstatě jen derivace tanh, dosazení do vzorečků a použití řetězového pravidla.

In [9]:
class RNN:
    def __init__(self, input_size, output_size, hidden_size=64):
        # Weights - the division is used for lowering the variance
        self.Whh = np.random.randn(hidden_size, hidden_size) / 1000
        self.Wxh = np.random.randn(hidden_size, input_size) / 1000
        self.Why = np.random.randn(output_size, hidden_size) / 1000

        # Biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
     
        
    # Processes whole sequence (not just one token / piece)
    def forward(self, inputs):
        # Variable for remembering the previous hidden state value
        h = np.zeros((self.Whh.shape[0], 1))

        self.last_inputs = inputs
        self.last_hs = { 0: h }

        # Gradual update of the hidden state
        for i, x in enumerate(inputs):
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)
            self.last_hs[i + 1] = h

        # Output vector computation
        y = np.dot(self.Why, h) + self.by

        return y, h
    
    
    def backprop(self, probs, target, learning_rate=2e-2):
        # Compute derivation of the loss function on the output (dL/dy)
        # (For now just believe me this is the right derivation of L(softmax(y)) w.r.t. y.)
        dLoss_wrt_Net_output = probs
        dLoss_wrt_Net_output[target] -= 1
        n = len(self.last_inputs)

        # Compute derivation of the loss function for output weights (dL/dWhy) and bias (dL/dby)
        dLoss_wrt_Weights_hidden_to_output = np.dot(dLoss_wrt_Net_output, self.last_hs[n].T)
        dLoss_wrt_Bias_of_output = dLoss_wrt_Net_output

        # Initialize zero matrices and vector for comuting dL/dWhh, dL/dWxh, and dL/dbh
        dLoss_wrt_Weights_hidden_to_hidden = np.zeros(self.Whh.shape)
        dLoss_wrt_Weights_input_to_hidden = np.zeros(self.Wxh.shape)
        dLoss_wrt_Bias_of_hidden = np.zeros(self.bh.shape)

        # Compute dL/dh for the last value of h
        dLoss_wrt_Hidden_activation = np.dot(self.Why.T, dLoss_wrt_Net_output)

        # Backpropagate the loss back in time by substitution into the equations
        for t in reversed(range(n)):
            # Derivative of the activation of a hidden layer wrt. input of the activation (dh/dhin)
            # dh/dhin = (1 - h^2) (Derivative of tanh(x) is (1 - tanh(x)^2))
            dHidden_activation_wrt_Hidden_activation_input = (1 - self.last_hs[t + 1] ** 2)
            
            # Gradient of loss wrt. input of the activation of a hidden layer (dL/dhin): dL/dh * dh/dhin
            dLoss_wrt_Hidden_activation_input = (dHidden_activation_wrt_Hidden_activation_input * dLoss_wrt_Hidden_activation)

            # dL/dbh = dL/dhin
            dLoss_wrt_Bias_of_hidden += dLoss_wrt_Hidden_activation_input

            # dL/dWhh = dL/dhin * h_{t-1}
            dLoss_wrt_Weights_hidden_to_hidden += np.dot(dLoss_wrt_Hidden_activation_input, self.last_hs[t].T)

            # dL/dWxh = dL/dhin * x
            dLoss_wrt_Weights_input_to_hidden += np.dot(dLoss_wrt_Hidden_activation_input, self.last_inputs[t].T)

            # dL/dh = dL/dhin * Whh
            dLoss_wrt_Hidden_activation = np.dot(self.Whh, dLoss_wrt_Hidden_activation_input)

        # To prevent too large gradients, we restrict / clip the values to interval [-1, 1]
        for gradient in [dLoss_wrt_Weights_input_to_hidden, dLoss_wrt_Weights_hidden_to_hidden, dLoss_wrt_Weights_hidden_to_output, dLoss_wrt_Bias_of_hidden, dLoss_wrt_Bias_of_output]:
            np.clip(gradient, -1, 1, out=gradient)

        # Update of weights and biases using gradient descent
        self.Whh -= learning_rate * dLoss_wrt_Weights_hidden_to_hidden
        self.Wxh -= learning_rate * dLoss_wrt_Weights_input_to_hidden
        self.Why -= learning_rate * dLoss_wrt_Weights_hidden_to_output
        self.bh -= learning_rate * dLoss_wrt_Bias_of_hidden
        self.by -= learning_rate * dLoss_wrt_Bias_of_output

In [10]:
def run_model(data, train=True):
    items = list(data.items())
    random.shuffle(items)

    loss = 0
    correct_answers = 0

    for x, y in items:
        inputs = create_inputs(x)
        target = int(y)

        # Forward pass
        out, _ = rnn.forward(inputs)
        probs = softmax(out)

        # Compute loss and accuracy
        loss -= np.log(probs[target])
        correct_answers += int(np.argmax(probs) == target)

        if train:
            # Backward pass
            rnn.backprop(probs, target)

    return loss / len(data), correct_answers / len(data)


np.random.seed(42)

rnn = RNN(vocab_size, 2)

for epoch in range(1001):
    train_loss, train_accuracy = run_model(train_data, train=True)
    if epoch % 100 == 0:
        print(f"--- Epoch {epoch:d}")
        print(f"Train:\tLoss {train_loss.item():3f} | Accuracy: {train_accuracy:3f}")

        test_loss, test_accuracy = run_model(test_data, train=False)
        print(f"Test:\tLoss {test_loss.item():3f} | Accuracy: {test_accuracy:3f}")

--- Epoch 0
Train:	Loss 0.695823 | Accuracy: 0.465517
Test:	Loss 0.693921 | Accuracy: 0.500000
--- Epoch 100
Train:	Loss 0.687747 | Accuracy: 0.551724
Test:	Loss 0.695497 | Accuracy: 0.500000
--- Epoch 200
Train:	Loss 0.669899 | Accuracy: 0.655172
Test:	Loss 0.716221 | Accuracy: 0.500000
--- Epoch 300
Train:	Loss 0.670781 | Accuracy: 0.568966
Test:	Loss 0.705595 | Accuracy: 0.500000
--- Epoch 400
Train:	Loss 0.325791 | Accuracy: 0.896552
Test:	Loss 0.302585 | Accuracy: 0.950000
--- Epoch 500
Train:	Loss 0.011523 | Accuracy: 1.000000
Test:	Loss 0.018818 | Accuracy: 1.000000
--- Epoch 600
Train:	Loss 0.005184 | Accuracy: 1.000000
Test:	Loss 0.008519 | Accuracy: 1.000000
--- Epoch 700
Train:	Loss 0.003350 | Accuracy: 1.000000
Test:	Loss 0.005411 | Accuracy: 1.000000
--- Epoch 800
Train:	Loss 0.002481 | Accuracy: 1.000000
Test:	Loss 0.003983 | Accuracy: 1.000000
--- Epoch 900
Train:	Loss 0.001958 | Accuracy: 1.000000
Test:	Loss 0.003105 | Accuracy: 1.000000
--- Epoch 1000
Train:	Loss 0.001

##  Sekvenční klasifikace pomocí LSTM

Nyní, když chápeme, jak taková základní RNN funguje, zkusíme se podívat na složitější druh RNN - LSTM sítě. Tyto sítě mají uvnitř sebe paměťovou buňku a navíc i mechanismus, který řídí, jakou informaci si buňka pamatuje a jakou zapomíná. (Pro podrobnější čtení o LSTM můžete jít na následující [odkaz](https://colah.github.io/posts/2015-08-Understanding-LSTMs/).) Zkusíme se na ni lépe podívat v následujícím příkladu sekvenční klasifikace. 

Sekvenční klasifikace je prediktivní modelovací problém, kdy máme na vstupu nějakou sekvenci v prostoru nebo čase a cílem je předpovědět kategorii této sekvence (jako tomu bylo třeba i v předchozím příkladu). Složitost tohoto problému spočívá v tom, že jednotlivé sekvence mohou mít různou délku nebo mohou být složeny z rozsáhlého slovníku vstupních hodnot a mohou vyžadovat, aby se model naučil nějaké dlouhodobé závislosti nebo kontext mezi vstupními sekvencemi.

Zkusíme se tedy podívat na příklad sekvenční klasifikace pomocí LSTM na IMDB datasetu, což je dataset, který obsahuje slovní popis recenzí 50K filmů a následně klasifikaci, jestli byla recenze pozitivní nebo negativní v poměru zhruba 1:1.

Problém je, že slovní popis je nějak potřeba převést na číselnou reprezentaci. Naštěstí funkce `imdb.load_data` umí načíst data tak, že rovnou slova nahradí čísly a rozdělí je na train a test množiny v poměru 1:1. Navíc data načteme tak, že necháme jen prvních `top_words` nejčastějších slov a zbytek nahradíme 0. Dále je potřeba zkrátit nebo doplnit vstupní sekvence pro modelování tak, aby byly všechny stejně dlouhé - délku nastavíme na `max_len`.

In [11]:
# Let us fix the random seed for the sake of reproducibility
np.random.seed(7)

In [12]:
top_words = 5000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=top_words)

max_length = 500
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_length)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_length)

In [13]:
x_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

Nyní máme připravená data a můžeme si definovat a natrénovat model.
 - První vrstva je `Embedding`, která každé slovo (v tuto chvíli nezáporné celé číslo reprezentující dané slovo) transformuje na vektor délky 32.
 - Další vrstva je `LSTM` vrstva, která obsahuje 100 paměťových jednotek (neuronů). 
 - Na závěr použijeme `Dense` výstupní vrstvu s jedním neuronem a aktivační funkcí sigmoid k vytvoření predikcí 0 nebo 1, protože se jedná o klasifikační úlohu.

Problém modelu je, že se velice snadno overfittuje na na daná trénovací data. Proto se se používají ještě vrstvy `Dropout`, které spočívají v tom, že během trénování se náhodně vynechávají některé vstupy do další vrstvy. Tím se simuluje velký počet sítí s odlišnou strukturou a uzly jsou pak robustnější a omezuje se tím pádem overfitting.

(Tohle je první místo, kde opravdu oceníte, používá-li váš tensorflow GPU. Pokud jste si tensorflow zprovoznili na GPU, schválně si následující kód zkuste spustit se zakázaným GPU (viz předchozí cvičení) a uvidíte ten rozdíl v době běhu.)

In [14]:
# Create the model
embedding_vector_length = 32
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(top_words, embedding_vector_length, input_length=max_length))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())
model.fit(x_train, y_train, epochs=3, batch_size=64)
pass

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 dropout (Dropout)           (None, 500, 32)           0         
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==========================

Na závěr zkusíme predikovat výstupy na testovacích datech a podívat se, jak je model dobrý. Můžete si třeba zkusit pustit trénování modelu s droupoutem a bez něj a podívat se, jak se budou lišit výsledné accuracies.

In [15]:
scores = model.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy: {scores[1]*100:.2f}%")

Accuracy: 86.40%


## Generování textu znak po znaku 

Nyní se podíváme na jiný druh problému - budeme generovat text znak po znaku, neboli natrénujeme jazykový model tak, že když mu pak dáme sekvenci znaků, tak nám model bude schopný předpovědět další znak. Jako trénovací množinu použijeme texty Nietzscheho.

In [ ]:
"""
    Example script to generate text from Nietzsche"s writings.
    At least 20 epochs are required before the generated text
    starts to sound coherent.
    It is recommended to run this script on GPU, as recurrent
    networks are quite computationally intensive.
    If you try this script on a new data, make sure your corpus
    has at least ~100k characters. ~1M is better.
"""

# Load the input data
path = tf.keras.utils.get_file("nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print("corpus length:", len(text))

chars = set(text)
print("total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Divide the text into partially dependent character sequences of length equal to maxlen
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print("nb sequences:", len(sentences))

# Convert text to numerical vectors
print("Vectorization...")
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
        
    y[i, char_indices[next_chars[i]]] = 1
    
# Create a model
print("Building model...")
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(512, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(len(chars), activation=tf.nn.softmax))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# Auxilary function for gaining index from an array of probabilities
def sample(a, temperature=1.0):  
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    a = a / np.sum(a)
    return np.argmax(np.random.multinomial(1, a, 1))


# Train model and generate output after each iteration
for iteration in range(1, 60):
    print()
    print("-" * 50)
    print("Iteration", iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print("----- diversity:", diversity)

        generated = ""
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print("----- Generating with seed: \"" + sentence + "\"")
        sys.stdout.write(generated)

        for _ in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Skript si samozřejmě můžete pustit, ale trénování poběží neskutečně dlouho. Proto jsme skript pustili na Google Colab a na výsledky se můžete podívat [zde](https://colab.research.google.com/drive/1B7zys275xmpPqahPwNvuYMPLmgvlV3l5) nebo v souboru [*results.txt*](results.txt).